In [18]:
import pandas as pd

# Load the CSV file
file_path = "modified_cause_progression.csv"
df = pd.read_csv(file_path)

# List of columns to keep
columns_to_keep = [
    "File Start Date", "File End Date", "File Start Time", "File End Time", 
    "File Duration", "Hectometer Head", "Hectometer Tail", "Route Letter", 
    "Route Number", "Route Description", "Hectometering Direction", 
    "Head Road Section From", "Head Road Section To", "Trajectory From", "Trajectory To"
]

# Drop all other columns except the ones listed above
df_filtered = df[columns_to_keep]

# Display the first few rows of the filtered dataframe
df_filtered.head()

,File Start Date,File End Date,File Start Time,File End Time,File Duration,Hectometer Head,Hectometer Tail,Route Letter,Route Number,Route Description,Hectometering Direction,Head Road Section From,Head Road Section To,Trajectory From,Trajectory To
0,2022-04-04,2022-04-04,08:02:29,08:09:00,"6,517","21,7","25,4",A,4,A4,descending,Roelofarendsveen,Ringvaart-Aquaduct,Den Haag,Amsterdam
1,2022-04-04,2022-04-04,08:02:29,08:10:02,"7,550","26,8","29,3",A,4,A4,descending,Hoogmade,Roelofarendsveen,Den Haag,Amsterdam
2,2022-04-04,2022-04-04,08:03:30,08:06:00,"2,500","53,0","56,2",A,16,A16,descending,Zonzeel,Zevenbergschen Hoek,Breda,Rotterdam
3,2022-04-04,2022-04-04,08:03:30,08:07:00,"3,500","31,0","32,9",A,27,A27,descending,Werkendam,Nieuwendijk,Gorinchem,Breda
4,2022-04-04,2022-04-04,08:03:30,08:42:03,"38,550","0,0","2,3",A,44,A44,descending,Nieuw-Vennep,Burgerveen,Wassenaar,Amsterdam


In [34]:
# Create a copy of the filtered DataFrame to avoid SettingWithCopyWarning
df_filtered_copy = df_filtered.copy()

# Converting time columns to datetime for easier filtering
df_filtered_copy['start_datetime'] = pd.to_datetime(df_filtered_copy['File Start Date']+df_filtered_copy['File Start Time'], format='%m-%d-%Y%H:%M:%S', errors='coerce').dt.time
df_filtered_copy['end_datetime'] = pd.to_datetime(df_filtered_copy['File End Date']+df_filtered_copy['File End Time'], format='%H:%M:%S', errors='coerce').dt.time

# Adding a new column to check if the start hour is different from the end hour
df_filtered_copy['Different Start-End Hour'] = df_filtered_copy['File Start Time'].apply(lambda x: x.hour) != df_filtered_copy['File End Time'].apply(lambda x: x.hour)

pd.set_option('display.max_rows', 10)
df_filtered_copy.head(10)

,File Start Date,File End Date,File Start Time,File End Time,File Duration,Hectometer Head,Hectometer Tail,Route Letter,Route Number,Route Description,Hectometering Direction,Head Road Section From,Head Road Section To,Trajectory From,Trajectory To,Different Start-End Hour
0,2022-04-04,2022-04-04,08:02:29,08:09:00,"6,517","21,7","25,4",A,4,A4,descending,Roelofarendsveen,Ringvaart-Aquaduct,Den Haag,Amsterdam,False
1,2022-04-04,2022-04-04,08:02:29,08:10:02,"7,550","26,8","29,3",A,4,A4,descending,Hoogmade,Roelofarendsveen,Den Haag,Amsterdam,False
2,2022-04-04,2022-04-04,08:03:30,08:06:00,"2,500","53,0","56,2",A,16,A16,descending,Zonzeel,Zevenbergschen Hoek,Breda,Rotterdam,False
3,2022-04-04,2022-04-04,08:03:30,08:07:00,"3,500","31,0","32,9",A,27,A27,descending,Werkendam,Nieuwendijk,Gorinchem,Breda,False
4,2022-04-04,2022-04-04,08:03:30,08:42:03,"38,550","0,0","2,3",A,44,A44,descending,Nieuw-Vennep,Burgerveen,Wassenaar,Amsterdam,False
5,2022-04-04,2022-04-04,08:20:30,08:25:01,"4,517","41,5","44,2",A,2,A2,descending,Vinkeveen,Abcoude,Utrecht,Amsterdam,False
6,2022-04-04,2022-04-04,08:20:30,08:58:00,"37,500","79,6","83,0",A,27,A27,descending,De Bilt,Rijnsweerd,Utrecht,Gorinchem,False
7,2022-04-04,2022-04-04,08:20:30,08:22:00,"1,500","19,1","21,1",A,58,A58,descending,Oirschot,Best,Tilburg,Eindhoven,False
8,2022-04-04,2022-04-04,08:20:30,08:51:01,"30,517","159,4","161,4",N,2,N2,descending,Eindhoven-Centrum,Eindhoven-Airport,Leenderheide,Batadorp,False
9,2022-04-04,2022-04-04,08:21:30,09:01:01,"39,517","52,3","54,3",A,15,A15,descending,Rotterdam-Heijplaat,Benelux,Ridderkerk,Europoort,True


In [35]:
df_filtered_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286069 entries, 0 to 286068
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   File Start Date           286069 non-null  object
 1   File End Date             286069 non-null  object
 2   File Start Time           286069 non-null  object
 3   File End Time             286069 non-null  object
 4   File Duration             286069 non-null  object
 5   Hectometer Head           286069 non-null  object
 6   Hectometer Tail           286069 non-null  object
 7   Route Letter              286069 non-null  object
 8   Route Number              286069 non-null  int64 
 9   Route Description         286069 non-null  object
 10  Hectometering Direction   286069 non-null  object
 11  Head Road Section From    286069 non-null  object
 12  Head Road Section To      286069 non-null  object
 13  Trajectory From           286069 non-null  object
 14  Traj

In [36]:
# Create a copy of the filtered DataFrame to avoid SettingWithCopyWarning
df_substract_time = df_filtered_copy.copy()

df_substract_time["Time Passed"] = df_substract_time['File End Time'] - df_substract_time['File Start Time']

df_substract_time

TypeError: unsupported operand type(s) for -: 'datetime.time' and 'datetime.time'